In [1]:
import pandas as pd
import requests
import pymongo
import os

from splinter import Browser
from bs4 import BeautifulSoup as bs
from pprint import pprint

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
db = client.space_db
collection = db.articles

In [6]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)

In [55]:
for x in range(1, 6):

    html = browser.html
    soup = bs(html, 'html.parser')
    
    posts = soup.find_all("div", class_="list_text")
    
    for post in posts:
        title = post.a.text
        para = post.find("div", class_="article_teaser_body").text
        
        print("------------------")
        print(title)
        print(para)
        
        article ={
            "news_title": title,
            "news_paragraph": para
        }

------------------
NASA to Host Media Call on Next Mars Landing Site
NASA will host a media teleconference at 9 a.m. PST (noon EST) Monday, Nov. 19, to provide details about the Mars 2020 rover’s landing site on the Red Planet.
------------------
How NASA Will Know When InSight Touches Down
On Nov. 26, engineers will look for a combination of signals to determine whether the next spacecraft to Mars lands safely.
------------------
NASA Brings Mars Landing to Viewers Everywhere
NASA's InSight lander is scheduled to touch down on the Red Planet at approximately noon PST  on Nov. 26, with a new suite of instruments to probe below the Martian surface.
------------------
Curiosity on the Move Again
NASA's Mars Curiosity rover drove about 197 feet over the weekend to a site called Lake Orcadie, pushing its total odometry to over 12 miles.
------------------
The Mars InSight Landing Site Is Just Plain Perfect
If the InSight landing zone were ice cream, it would be vanilla.
------------------


------------------
NASA to Host Media Call on Next Mars Landing Site
NASA will host a media teleconference at 9 a.m. PST (noon EST) Monday, Nov. 19, to provide details about the Mars 2020 rover’s landing site on the Red Planet.
------------------
How NASA Will Know When InSight Touches Down
On Nov. 26, engineers will look for a combination of signals to determine whether the next spacecraft to Mars lands safely.
------------------
NASA Brings Mars Landing to Viewers Everywhere
NASA's InSight lander is scheduled to touch down on the Red Planet at approximately noon PST  on Nov. 26, with a new suite of instruments to probe below the Martian surface.
------------------
Curiosity on the Move Again
NASA's Mars Curiosity rover drove about 197 feet over the weekend to a site called Lake Orcadie, pushing its total odometry to over 12 miles.
------------------
The Mars InSight Landing Site Is Just Plain Perfect
If the InSight landing zone were ice cream, it would be vanilla.
------------------


------------------
NASA's MAVEN Spacecraft Finds That 'Stolen' Electrons Enable Unusual Aurora on Mars
Auroras appear on Earth as ghostly displays of colorful light in the night sky, usually near the poles.
------------------
'Storm Chasers' on Mars Searching for Dusty Secrets
Scientists with NASA's Mars orbiters have been waiting years for an event like the current Mars global dust storm.
------------------
NASA Mars Mission Adds Southern California Dates
Looking for summer fun? Southern California families have their choice of the beach, movies, museums -- and even NASA's next mission to Mars.
------------------
Curiosity Captures Photos of Thickening Dust
A storm of tiny dust particles has engulfed much of Mars over the last two weeks.
------------------
NASA Encounters the Perfect Storm for Science
One of the most intense Martian dust storms ever observed is being studied by a record number of NASA spacecraft.
------------------
Media Telecon About Mars Dust Storm, Opportunity
NASA

------------------
The 'Claw Game' on Mars: NASA InSight Plays to Win
NASA's InSight lander will be the first mission to use a robotic arm to grasp instruments from the spacecraft and place them on the surface of another planet. 
------------------
Scientists to Debate Landing Site for Next Mars Rover
Hundreds of scientists and Mars-exploration enthusiasts gather this week to present, discuss and deliberate the future landing site for NASA's next Red Planet. 
------------------
Mars Virtual Reality Software Wins NASA Award
A mixed-reality software that allows scientists and engineers to virtually walk on Mars has received NASA's 2018 Software of the Year Award.
------------------
Painting Cars for Mars
What does NASA do to get a rover ready for Mars? JPL's paint shop does a thousand different paint jobs a year. And while every piece of spacecraft that comes through the shop is important, one headed for the Red Planet adds a little extra oomph.
------------------
Opportunity Emerges in 